In [ ]:
!pip install opencv-python numpy scipy tqdm

In [ ]:
import os
import glob
import argparse
import numpy as np
import cv2
from tqdm import tqdm
from scipy.spatial import distance
from scipy.ndimage.filters import convolve
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import spsolve

<ipython-input-2-f2d4de94e4db>:8: DeprecationWarning: Please import `convolve` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import convolve


In [ ]:
def get_sparse_neighbor(p: int, n: int, m: int):
    """Returns a dictionary of 4-neighbors of `p` in a sparse matrix."""
    i, j = p // m, p % m
    d = {}
    if i - 1 >= 0:
        d[(i - 1) * m + j] = (i - 1, j, 0)
    if i + 1 < n:
        d[(i + 1) * m + j] = (i + 1, j, 0)
    if j - 1 >= 0:
        d[i * m + j - 1] = (i, j - 1, 1)
    if j + 1 < m:
        d[i * m + j + 1] = (i, j + 1, 1)
    return d


In [ ]:
def create_spacial_affinity_kernel(spatial_sigma: float, size: int = 15):
    kernel = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            kernel[i, j] = np.exp(-0.5 * (distance.euclidean((i, j), (size // 2, size // 2)) ** 2) / (spatial_sigma ** 2))
    return kernel


def compute_smoothness_weights(L, x, kernel, eps=1e-3):
    Lp = cv2.Sobel(L, cv2.CV_64F, int(x == 1), int(x == 0), ksize=1)
    T = convolve(np.ones_like(L), kernel, mode='constant')
    T = T / (np.abs(convolve(Lp, kernel, mode='constant')) + eps)
    return T / (np.abs(Lp) + eps)


def refine_illumination_map_linear(L, gamma, lambda_, kernel, eps=1e-3):
    wx = compute_smoothness_weights(L, x=1, kernel=kernel, eps=eps)
    wy = compute_smoothness_weights(L, x=0, kernel=kernel, eps=eps)

    n, m = L.shape
    L_1d = L.copy().flatten()

    row, column, data = [], [], []
    for p in range(n * m):
        diag = 0
        for q, (k, l, x) in get_sparse_neighbor(p, n, m).items():
            weight = wx[k, l] if x else wy[k, l]
            row.append(p)
            column.append(q)
            data.append(-weight)
            diag += weight
        row.append(p)
        column.append(p)
        data.append(diag)
    F = csr_matrix((data, (row, column)), shape=(n * m, n * m))

    Id = diags([np.ones(n * m)], [0])
    A = Id + lambda_ * F
    L_refined = spsolve(csr_matrix(A), L_1d, permc_spec=None, use_umfpack=True).reshape((n, m))

    return np.clip(L_refined, eps, 1) ** gamma


def correct_underexposure(im, gamma, lambda_, kernel, eps=1e-3):
    L = np.max(im, axis=-1)
    L_refined = refine_illumination_map_linear(L, gamma, lambda_, kernel, eps)
    L_refined_3d = np.repeat(L_refined[..., None], 3, axis=-1)
    return im / L_refined_3d


def enhance_image_exposure(im, gamma, lambda_, dual=True, sigma=3, bc=1, bs=1, be=1, eps=1e-3):
    kernel = create_spacial_affinity_kernel(sigma)
    im_normalized = im.astype(float) / 255.
    under_corrected = correct_underexposure(im_normalized, gamma, lambda_, kernel, eps)

    if dual:
        inv_im_normalized = 1 - im_normalized
        over_corrected = 1 - correct_underexposure(inv_im_normalized, gamma, lambda_, kernel, eps)
        merge_mertens = cv2.createMergeMertens(bc, bs, be)
        images = [np.clip(x * 255, 0, 255).astype("uint8") for x in [im_normalized, under_corrected, over_corrected]]
        im_corrected = merge_mertens.process(images)
    else:
        im_corrected = under_corrected

    return np.clip(im_corrected * 255, 0, 255).astype("uint8")


In [ ]:
def process_images(args):
    imdir = args.folder
    if not os.path.exists(imdir):
        os.makedirs(imdir)

    ext = ['png', 'jpg', 'bmp']
    files = []
    [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
    images = [cv2.imread(file) for file in files]

    directory = os.path.join(imdir, "/content/enhanced")
    if not os.path.exists(directory):
        os.makedirs(directory)

    for i, image in tqdm(enumerate(images), desc="Enhancing images"):
        enhanced_image = enhance_image_exposure(image, args.gamma, args.lambda_, not args.lime,
                                                sigma=args.sigma, bc=args.bc, bs=args.bs, be=args.be, eps=args.eps)
        filename = os.path.basename(files[i])
        # name, ext = os.path.splitext(filename)
        # method = "LIME" if args.lime else "DUAL"
        # corrected_name = f"{name}_{method}_g{args.gamma}_l{args.lambda_}{ext}"
        cv2.imwrite(os.path.join(directory, filename), enhanced_image)


In [ ]:
args = argparse.Namespace(
    folder="/content/low/",
    gamma=0.6,
    lambda_=0.15,
    lime=False,
    sigma=3,
    bc=1,
    bs=1,
    be=1,
    eps=1e-3
)

process_images(args)


Enhancing images: 15it [02:24,  9.65s/it]


# Displaying the output images


In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt

In [ ]:
def load_image(image_path):
    """Load an image and convert it to RGB format for correct display in Matplotlib."""
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return None

    image = cv2.imread(image_path)  # Read image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    return image

In [ ]:
def display_images(input_path, enhanced_path, high_path):
    """Display input and enhanced images side by side."""
    input_image = load_image(input_path)
    enhanced_image = load_image(enhanced_path)
    high_image = load_image(high_path)

    if input_image is None or enhanced_image is None or high_image is None:
        return

    # Create a side-by-side plot
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    ax[0].imshow(input_image)
    ax[0].set_title("Low Light Image")
    ax[0].axis("off")

    ax[1].imshow(enhanced_image)
    ax[1].set_title("Enhanced Image")
    ax[1].axis("off")

    ax[2].imshow(high_image)
    ax[2].set_title("Ground Truth")
    ax[2].axis("off")

    plt.show()

In [ ]:
def process_and_display_images(input_folder, output_folder, high_folder):
    """Find input images in input_folder, locate their enhanced versions in output_folder, and display them."""

    ext = ['png', 'jpg', 'bmp']
    input_files = []
    [input_files.extend(glob.glob(os.path.join(input_folder, f'*.{e}'))) for e in ext]

    if not input_files:
        print("No images found in the input folder.")
        return

    for input_file in input_files:
        filename = os.path.basename(input_file)

        # Look for the enhanced image in the output folder
        enhanced_file = os.path.join(output_folder, filename)

        high_file = os.path.join(high_folder, filename)

        if os.path.exists(enhanced_file) and os.path.exists(high_file):
            display_images(input_file, enhanced_file, high_file)
        else:
            print(f"Enhanced image or high-quality image not found for {filename}")


In [ ]:
process_and_display_images(input_folder="/content/low/", output_folder="/content/enhanced/", high_folder="/content/high/")

Enhanced image or high-quality image not found for 14.png
Enhanced image or high-quality image not found for 7.png
Enhanced image or high-quality image not found for 11.png
Enhanced image or high-quality image not found for 2.png
Enhanced image or high-quality image not found for 4.png
Enhanced image or high-quality image not found for 1.png
Enhanced image or high-quality image not found for 5.png
Enhanced image or high-quality image not found for 6.png
Enhanced image or high-quality image not found for 15.png
Enhanced image or high-quality image not found for 8.png
Enhanced image or high-quality image not found for 10.png
Enhanced image or high-quality image not found for 9.png
Enhanced image or high-quality image not found for 13.png
Enhanced image or high-quality image not found for 3.png
Enhanced image or high-quality image not found for 12.png


# Performance Evaluation Techniques

MSE : Mean Square Error
Calculating the MSE of the High light image wuth the enhanced image

In [ ]:
!pip install opencv-python numpy scipy tqdm pandas scikit-image

In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

PSNR : Peak Signal to Noise Ratio

In [ ]:
# Function to compute PSNR
def calculate_psnr(ref, img):
    """
    Calculate the Peak Signal-to-Noise Ratio (PSNR) between two images.

    Parameters:
        ref: The first image as a NumPy array.
        img: The second image as a NumPy array.

    Returns:
        psnr: The PSNR value in decibels (dB).
    """
    mse = np.mean((ref - img) ** 2)
    if mse == 0:
        return 100  # Perfect match
    max_pixel = 255.0
    return 10 * np.log10((max_pixel ** 2) / mse)

SSIM : structural similarity index measure

In [ ]:
# Function to compute SSIM
def calculate_ssim(ref, img):
    """
    Calculate the Structural Similarity Index (SSIM) between two images.

    Parameters:
        ref: The first image as a NumPy array.
        img: The second image as a NumPy array.

    Returns:
        ssim_value: The SSIM value (ranges from -1 to 1).
    """
    return ssim(ref, img, channel_axis=-1)

Function to Evaluate All Images in a Folder

In [ ]:
def evaluate_images(low_folder, enhanced_folder, high_folder):
    """Evaluate image quality metrics for all images in a folder and compute average results."""

    ext = ['png', 'jpg', 'bmp']
    low_images = []
    [low_images.extend(glob.glob(os.path.join(low_folder, f'*.{e}'))) for e in ext]

    if not low_images:
        print("No images found in the low-light folder.")
        return

    results = []

    for low_file in low_images:
        filename = os.path.basename(low_file)

        # Find corresponding enhanced image
        enhanced_file = os.path.join(enhanced_folder, filename)  # Adjust pattern if needed
        # Find corresponding high-quality (ground truth) image
        high_file = os.path.join(high_folder, filename)

        if os.path.exists(enhanced_file) and os.path.exists(high_file):
            # Load images
            enhanced_img = cv2.imread(enhanced_file)
            high_img = cv2.imread(high_file)

            # Compute quality metrics
            psnr_value = calculate_psnr(high_img, enhanced_img)
            ssim_value = calculate_ssim(high_img, enhanced_img)

            # Store results
            results.append({"Image": filename, "PSNR (dB)": psnr_value, "SSIM": ssim_value})
        else:
            print(f"Matching image not found for {filename}")

    # Convert results to a DataFrame
    df = pd.DataFrame(results)

    # Compute the average values
    avg_psnr = df["PSNR (dB)"].mean()
    avg_ssim = df["SSIM"].mean()

    print("\n📊 **Evaluation Results:**")
    print(df)
    print("\n📌 **Overall Average Metrics:**")
    print(f"🔹 Average PSNR: {avg_psnr:.4f} dB")
    print(f"🔹 Average SSIM: {avg_ssim:.4f}")


In [ ]:
evaluate_images(low_folder="/content/low/", enhanced_folder="/content/enhanced/", high_folder="/content/high/")


📊 **Evaluation Results:**
     Image  PSNR (dB)      SSIM
0   14.png  27.809406  0.469826
1    7.png  28.142979  0.633399
2   11.png  27.527465  0.336018
3    2.png  27.567716  0.557659
4    4.png  28.619063  0.297663
5    1.png  27.653034  0.634395
6    5.png  27.466599  0.552484
7    6.png  27.502728  0.535939
8   15.png  28.827468  0.523558
9    8.png  30.300109  0.746125
10  10.png  27.813502  0.546906
11   9.png  28.813358  0.543278
12  13.png  27.675230  0.589216
13   3.png  28.628661  0.315303
14  12.png  27.515978  0.500856

📌 **Overall Average Metrics:**
🔹 Average PSNR: 28.1242 dB
🔹 Average SSIM: 0.5188
